# 시그모이드 함수 그리기

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

- 주어진 데이터에 가장 적합한 w와 b를 구하는 것이 목적이다.
- x가 0일 때 출력 값이 0.5를 갖는다.

In [ ]:
x = np.arange(-5.0, 5.0, 0.1)
y = sigmoid(x)

plt.plot(x, y, 'g')
plt.plot([0,0],[1.0, 0.0],':') # 가운데 점선 추가
plt.title('Sigmoid Function')
plt.show()

- w의 값에 따라 경사도가 바뀐다. (선형회귀에서 w가 직선의 기울기를 의미하는 것과 동일)

In [ ]:
x = np.arange(-5.0, 5.0, 0.1)
y1 = sigmoid(0.5*x)
y2 = sigmoid(x)
y3 = sigmoid(2*x)

plt.plot(x, y1, 'r--', label='w:0.5')  # w가 0.5일 때
plt.plot(x, y2, 'g', label='w:1')    # w가 1일 때
plt.plot(x, y3, 'b--', label='w:2')    # w가 2일 때

plt.plot([0,0],[1.0, 0.0],':') # 가운데 점선 추가
plt.title('Sigmoid Function')
plt.legend()
plt.show()

# 유방암 판별 예측
- y변수가 범주형인 경우

## 패키지 로딩

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd


## 데이터 로드 및 확인

In [ ]:
breast = load_breast_cancer()
# print(breast.DESCR)

df = pd.DataFrame(breast.data, columns=breast.feature_names)
df.head()
df['class'] = breast.target
df.head()
df['class'].value_counts()
df.describe()

## 독립변수 / 종속변수 선택

In [ ]:
X = breast.data
y = breast.target

X.shape, y.shape

## 데이터 스케일링

In [ ]:
scaled_X = StandardScaler().fit_transform(X)

## 학습 데이터와 평가 데이터 분할

In [ ]:
# stratify 종속변수 비율에 맞춰서 분배 
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.3, random_state=0, stratify=y)

## 모델 생성
- 회귀계수 최적화 옵션
    - solver : 최적화 문제에 사용할 알고리즘
        >- 'lbfgs' : solver의 기본값. CPU 코어 수가 많다면 최적화를 병렬로 사용할 수 있다
        >- 'liblinear' : 작은 데이터에 적합한 알고리즘.
        >- 'sag','saga' : 확률적 경사하강법을 기반으로 한 알고리즘으로 대용향 데이터에 적합
        >- 'newton-cg','sag','saga' 및 'libfgs'만 다항 손실을 처리. 즉, 멀티 클래스 분류 모델에 사용 가능
    - solver에 따른 규제 지원 사항
        >- newton-cg, lbfgs, sag : L2
        >- linlinear, saga : L1, L2
    - multi_class : 다중클래스분류 문제의 상황에서 어떤 접근방식을 취할지 결정
        >- 'ovr' : 이진분류기인 sigmoid 함수를 이용하여 결과 예측
        >- 'multinominal' : 각 클래스에 대한 softmax 확률 값으로 다중분류를 수행
    - C : 규제의 강도의 역수. 양의 실수 값. 선형회귀모형에서 alpha이고, SVM과 LogisticRegression과 같이 분류 모델에서는 C를 사용.(C값이 작을수록 모델이 단순해진다.)
    - max_iter : solver가 수렴하는데 필요한 반복 횟수 (default:100)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

print('추정 계수(가중치):', model.coef_)
print('절편:', model.intercept_)

## 모델 예측

In [ ]:
y_hat = model.predict(X_test)
print('예측값:', y_hat[:20])
print('실제값:', y_test[:20])

## 평가 지표
- 혼동 행렬 함수는 행을 true, 열을 predict 값으로 이용하며, 양성과 음성의 구분은 별도의 레이블을 지정하지 않으면 레이블 값의 정렬된 순서로 사용한다.(0:Negative, 1:Positive)

<pre>
y_true = [1,0,1,1,0,1]
t_hat =  [0,0,1,1,0,1]
confusion_matrix(y_true, y_hat)
[[2,0],
 [1,3]]
                  Predict
            ----------------
                |  N  |   P
            ----------------
   |        | N |  TN |  FP
   |  True  |---------------
   |        | P |  FN |  TP
   
confusion_matrix(y_true, y_hat, label=[1,0])
</pre>

In [ ]:
matrix = confusion_matrix(y_test, y_hat)
print(matrix)

In [ ]:
accuracy = accuracy_score(y_test, y_hat)
print(f'정확도:{accuracy:.3f}') #(61 + 103) / (61 + 3 + 4 + 103)

precision = precision_score(y_test, y_hat)
print(f'정밀도:{precision:.3f}') # 103 / ( 3 + 103)

recall = recall_score(y_test, y_hat)
print(f'재현율:{recall:.3f}')  # 103 / ( 4 + 103)

# [ N으로 예측할 확률, P로 예측할 확률]
pred_proba_positive = model.predict_proba(X_test)[:,1]
np.round(pred_proba_positive, 3)

# fpr, tpr, threshold 값 반환
fpr, tpr, threshold = roc_curve(y_test, pred_proba_positive)
roc_data = np.concatenate([fpr.reshape(-1,1), tpr.reshape(-1,1), np.round(threshold.reshape(-1,1),3)], axis=1)
roc_data = pd.DataFrame(roc_data, columns=['FPR','TPR','THRESHOLD'])
display(roc_data)
auc = roc_auc_score(y_test, pred_proba_positive)
print(f'AUC:{auc:.3f}')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(fpr, tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

In [ ]:
optimal_threshold = threshold[np.argmax(tpr - fpr)]
print('최적의 임계값:',optimal_threshold)

## 임계값 변화에 따른 재현율과 정밀도 변환 확인

In [ ]:
from sklearn.metrics import classification_report

def threshold_filter(prob, threshold):
    return np.where(prob > threshold, 1, 0)

pred_1 = threshold_filter(pred_proba_positive, 0.5)
pred_2 = threshold_filter(pred_proba_positive, 0.6)
pred_3 = threshold_filter(pred_proba_positive, 0.8)

print(classification_report(y_test, pred_1))
print('-'*100)
print(classification_report(y_test, pred_2))
print('-'*100)
print(classification_report(y_test, pred_3))


## solver별 성능평가 비교

In [ ]:
solvers = ['lbfgs','liblinear','newton-cg','sag','saga']

for solver in solvers:
    model = LogisticRegression(solver=solver, max_iter=600)
    model.fit(X_test, y_test)
    y_hat = model.predict(X_test)
    
    print(f'solver:{solver}, accuracy:{accuracy_score(y_test, y_hat):.3f}, roc_auc:{roc_auc_score(y_test, model.predict_proba(X_test)[:,1]):.3f}')

# [실습] 개인 신용도를 기반으로 대출 가능여부 예측하기
- 데이터 구조
    - Age : 나이
    - Experience : 경력
    - Income : 연소득
    - ZIP Code : 우편번호
    - Family : 가족 수
    - CCAvg : 월 평균 카드 사용액
    - Education : 교육수준 (1:학부, 2:대학원, 3:고급/전문)
    - Mortgage : 가계대출
    - Personal Loan : 대출 승인여부
    - Securities Account : 유가증권 계좌유무
    - CD Account : 양도예금증서 계좌 유무
    - Online : 온라인계좌 유무
    - CreditCard : 신용카드 유무
- 처리 조건 : 불필요한 컬럼삭제 (ID, Zip Code)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:
# 데이터 로드 및 확인
df = pd.read_csv('dataset/Personal_Loan.csv')
df.head()


# 데이터 전처리
df = df.drop(['ID','ZIP Code'], axis=1)


# 독립변수와 종속변수 분리
X = df.drop('Personal Loan', axis=1)
y = df['Personal Loan']


# 학습데이터와 평가데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40, stratify=y)


# 모델 생성
model = LogisticRegression(max_iter=2000)

# 모델 학습
model.fit(X_train, y_train)


np.set_printoptions(suppress=True, precision=2)

# 추정계수
coef = model.coef_.squeeze(axis=0)
print('추정계수:', coef)

# 밑에 설명있음
odds_rate = np.exp(coef)
print('추정계수(odds비):', odds_rate)

coef_df = pd.DataFrame({'가중치':coef, 'Odds비':odds_rate}, index=X.columns)
display(coef_df)


# 모델 예측
y_hat = model.predict(X_test)


# 모델 평가
cf = confusion_matrix(y_test, y_hat)
cf_df = pd.DataFrame(cf, index=[['actual','actual'],['대출불가(0)','대출승인(1)']], columns=[['predict','predict'],['대출불가(0)','대출승인(1)']])
display(cf_df)

print(f'정확도:{accuracy_score(y_test, y_hat):.3f}') 
print(f'정밀도:{precision_score(y_test, y_hat):.3f}') 
print(f'재현율:{recall_score(y_test, y_hat):.3f}')  


# [ N으로 예측할 확률, P로 예측할 확률]
pred_proba_positive = model.predict_proba(X_test)[:,1]

fpr, tpr, threshold = roc_curve(y_test, pred_proba_positive)

plt.plot(fpr, tpr)
plt.xlabel('FalsePositiveRate')
plt.ylabel('TruePositiveRate')
plt.show()

auc = roc_auc_score(y_test, pred_proba_positive)
print(f'AUC:{auc:.3f}')

- 로지스틱 회귀에서 회귀계수의 해석
    - 로지스틱 회귀계수는 지수변환(exp()함수를 취한다)을 해주면 Odds비가 나온다.
    - Odds비: 성공할 확률 / 실패할 확률
    - 대출여부에 영향을 미치는 a변수의 오즈비가 1보다 큰 경우 대출승인확률이 높다는 결과이다.(만약 14라면 대출거부확률보다 대출승인확률이 14배 높다는 의미)

## 교차검증


In [46]:
from sklearn.model_selection import cross_validate

scores = cross_validate(model, X, y, cv=10, scoring =['accuracy','precision','roc_auc'])

for key , value in scores.items():
    print('평가지표:', key)
    print('평균값:', np.round(np.mean(value),3))
    print('-'*50)


평가지표: fit_time
평균값: 0.504
--------------------------------------------------
평가지표: score_time
평균값: 0.007
--------------------------------------------------
평가지표: test_accuracy
평균값: 0.95
--------------------------------------------------
평가지표: test_precision
평균값: 0.81
--------------------------------------------------
평가지표: test_roc_auc
평균값: 0.958
--------------------------------------------------


## 학습모델 저장

In [47]:
import joblib # 모델 학습 결과를 pickle 파일로 저장하기 위한 라이브러리

joblib.dump(model, './lr_model.pkl')

['./lr_model.pkl']

In [49]:
loaded_model = joblib.load('./lr_model.pkl')
accuracy = loaded_model.score(X_test,y_test)
print(accuracy)

0.9553333333333334
